In [14]:
from ultralytics import YOLO
import numpy as np
from collections import Counter
model = YOLO(r"/Users/sujalpatel/Desktop/Traffic_Management_System/IOT-based-management-system/best.pt")


threshold = 0.001 
results = model.predict(source=r"/Users/sujalpatel/Desktop/Traffic_Management_System/IOT-based-management-system/test.jpg", imgsz=448, conf=threshold)



image 1/1 /Users/sujalpatel/Desktop/Traffic_Management_System/IOT-based-management-system/test.jpg: 320x448 3 buss, 68 cars, 10 motorbikes, 7 persons, 38.6ms
Speed: 0.8ms preprocess, 38.6ms inference, 0.8ms postprocess per image at shape (1, 3, 320, 448)


In [17]:
xyxy = results[0].boxes.xyxy.tolist()
conf = results[0].boxes.conf.tolist()
print(len(xyxy))

88


In [18]:

xyxy = results[0].boxes.xyxy.tolist()  # Bounding box coordinates
conf = results[0].boxes.conf.tolist()  # Confidence scores
cls = results[0].boxes.cls.tolist()    # Class labels

# Print the number of detected objects
print("Number of detected objects:", len(xyxy))

# Get class names based on the index (assuming you have the class names)
class_names = model.names  # This gives you the class names in a list
class_labels = [class_names[int(label)] for label in cls]

# Count the occurrences of each class label
category_counts = Counter(class_labels)

# Convert Counter to dictionary
category_counts_dict = dict(category_counts)
del category_counts_dict['person']
# Print the dictionary of category counts
print("Category counts:", category_counts_dict)


Number of detected objects: 88
Category counts: {'car': 68, 'motorbike': 10, 'bus': 3}


In [19]:
# Define PCU values for different vehicle types
PCU_VALUES = {
    'car': 0.7,
    'motorbike': 0.5,
    'bus': 1.5,
}

# Define minimum and maximum green light durations (in seconds)
T_MIN = 10  # minimum green light time
T_MAX = 60  # maximum green light time

# Example input: total count of each type of vehicle at the intersection
vehicles_at_intersection = category_counts_dict

def calculate_total_pcu(vehicles):
    """
    Calculate the total PCU for all vehicles at the intersection.
    """
    total_pcu = sum(count * PCU_VALUES.get(vehicle_type, 0) for vehicle_type, count in vehicles.items())
    return total_pcu

def calculate_green_time(total_pcu):
    """
    Calculate the green light duration based on total PCU proportion.
    """
    # Scale green light time based on traffic volume (PCU)
    if total_pcu > 0:
        return T_MIN + (total_pcu / (total_pcu + 10)) * (T_MAX - T_MIN)
    else:
        return T_MIN  # Default to minimum time if no vehicles

def calculate_waiting_time(total_pcu, average_wait_time_per_vehicle):
    """
    Estimate total waiting time for all vehicles at the intersection.
    """
    return total_pcu * average_wait_time_per_vehicle

# Calculate the total PCU for the intersection
total_pcu = calculate_total_pcu(vehicles_at_intersection)

# Calculate the dynamic green light duration based on the total PCU
green_time = calculate_green_time(total_pcu)

# Assume an average delay per vehicle based on traffic flow data (this can be dynamic in a real-time system)
average_wait_time_per_vehicle = 2  # in seconds

# Calculate estimated total waiting time at the intersection
total_waiting_time = calculate_waiting_time(total_pcu, average_wait_time_per_vehicle)

# Display results
print("Green light duration (in seconds):")
print(f"Total green time: {green_time:.2f} seconds")

print("\nEstimated total waiting time (in seconds):")
print(f"Total waiting time: {total_waiting_time:.2f} seconds")


Green light duration (in seconds):
Total green time: 52.55 seconds

Estimated total waiting time (in seconds):
Total waiting time: 114.20 seconds


In [ ]:
from ultralytics import YOLO
import numpy as np
from collections import Counter

# Define PCU values for different vehicle types
PCU_VALUES = {
    'car': 1.0,
    'motorbike': 0.5,
    'bus': 2.5
}

# Define minimum and maximum green light durations (in seconds)
T_MIN = 10  # minimum green light time
T_MAX = 60  # maximum green light time

# Path to the model and the images for each signal
model = YOLO(r"/Users/sujalpatel/Desktop/Traffic_Management_System/IOT-based-management-system/best.pt")
image_paths = {
    'signal_1': r"/Users/sujalpatel/Desktop/Traffic_Management_System/IOT-based-management-system/download.jpg",
    'signal_2': r"/Users/sujalpatel/Desktop/Traffic_Management_System/IOT-based-management-system/download.jpg",
    'signal_3': r"/Users/sujalpatel/Desktop/Traffic_Management_System/IOT-based-management-system/download.jpg",
    'signal_4': r"/Users/sujalpatel/Desktop/Traffic_Management_System/IOT-based-management-system/download.jpg"
}

def calculate_pcu(vehicles):
    """Calculate the total PCU for a given set of vehicles."""
    return sum(count * PCU_VALUES.get(vehicle_type, 0) for vehicle_type, count in vehicles.items())

def calculate_green_time(total_pcu, signal_pcu):
    """Calculate the green light duration for a signal based on its PCU proportion."""
    return T_MIN + (signal_pcu / total_pcu) * (T_MAX - T_MIN) if total_pcu > 0 else T_MIN

def calculate_waiting_time(signal_pcu, average_wait_time_per_vehicle=2):
    """Estimate the waiting time for a signal based on its total PCU."""
    return signal_pcu * average_wait_time_per_vehicle

def get_vehicle_counts(image_path):
    """Detect vehicles in an image and return their counts by category."""
    threshold = 0.001
    results = model.predict(source=image_path, imgsz=448, conf=threshold)
    
    # Count detected objects
    cls = results[0].boxes.cls.tolist()
    class_names = model.names  # Class names list
    class_labels = [class_names[int(label)] for label in cls]
    category_counts = Counter(class_labels)
    
    # Filter out non-vehicle classes (if needed)
    vehicle_counts = {k: category_counts.get(k, 0) for k in PCU_VALUES.keys()}
    return vehicle_counts

# Process each signal's image and calculate green time and waiting time
signal_data = {}
for signal, image_path in image_paths.items():
    vehicle_counts = get_vehicle_counts(image_path)
    signal_pcu = calculate_pcu(vehicle_counts)
    signal_data[signal] = {
        'vehicle_counts': vehicle_counts,
        'pcu': signal_pcu
    }

# Calculate total PCU across all signals
total_pcu = sum(data['pcu'] for data in signal_data.values())

# Calculate green light duration and waiting time for each signal
for signal, data in signal_data.items():
    green_time = calculate_green_time(total_pcu, data['pcu'])
    waiting_time = calculate_waiting_time(data['pcu'])
    
    print(f"{signal}:\n - Green Light Duration: {green_time:.2f} seconds\n - Waiting Time: {waiting_time:.2f} seconds\n")